In [2]:
from collections import defaultdict
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

MIN_BUSINESS_CATEGORY = 15

cwd = os.getcwd()
business_clean_path = cwd+"/a11_cleaned_businesses.json"
reviews_clean_path = cwd+"/a1_filtered_reviewsv3.json"

business = pd.read_json(business_clean_path, lines=True)

## En este paso del proceso, el objetivo es obtener el dataset final que concatena los perfiles del usuario con los lugares a los que ha hecho review y adicionalmente el target (rating).

### Este dataset sí incluye los pesos nulos/ceros (0) de las categorias en todos los perfiles. Por lo tanto debe ser comprimido usando parquet. 

In [3]:
# Esto pesa más de 500Mb
reviews = pd.read_json(reviews_clean_path, lines=True)

reviews.head()

review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   
4  pUycOfUwM8vqX7KjRRhUEA  59MxRhNVhU9MYndMkz0wtw  gebiRewfieSdtt17PTW6Zg   

   stars  
0      3  
1      5  
2      3  
3      4  
4      3

In [4]:
len(reviews)

4097564

### Creación del dataset de entrenamiento y validación
<p style="font-size:18px">
    • La idea aquí es unir los perfiles de los usuarios y de los lugares a los cuales han hecho reseñas y concatenar al final la columna que sería nuestro target: El puntaje de 1 a 5 de la review.<br>
    • El resultado será un dataset de aproximadamente 4 millones de filas y 1967 columnas <b>(1966 features| 983 categorias por perfil, 1 target).</b><br>
    • El dataset tendrá una gran cantidad de ceros (sparse matrices).<br>
    • 4M x 1966 x 4 bytes (float32) ≈ 31GB
</p>

In [7]:
#Probemos primero con todos los datos a ver si esta mrda procesa:
#---- Reconstruimos el dataset usando las categorias válidas ordenadas alfabéticamente ----
business["categories"] = business["categories"].str.split(", ")

category_counts = defaultdict(int)
for cats in business["categories"]:
    if isinstance(cats, list):
        for cat in cats:
            category_counts[cat] += 1


valid_categories = {cat for cat, count in category_counts.items() 
                    if count >= MIN_BUSINESS_CATEGORY}

print(f"Categorias finales {len(valid_categories)}")

Categorias finales 983


In [8]:
valid_categories = sorted(valid_categories)
valid_categories[0:5]

['& Probates', 'Acai Bowls', 'Accessories', 'Accountants', 'Acne Treatment']

In [13]:
user_profiles = json.load(open("a1_users_profile_l2_normalized.json"))
business_profiles = json.load(open("business_profiles_l2_normalized.json"))

# Create mappings for quick lookup
user_profile_map = {u["user_id"]: u["profile"] for u in user_profiles}
business_profile_map = {b["business_id"]: b["profile"] for b in business_profiles}

In [19]:
# Concatenar los perfiles
## ---- ❤ Gemini|Deepseek: ❤  ---- 

# Aun con 32Gb de RAM, python consumia toda mi memoria al intentar hacer la union de los perfiles completa. Se optó por hacerlo por batches
# Guardandolo primero en diferentes archivos parquet y luego, concatenandolos todos en uno grande.
from tqdm import tqdm


def process_chunks(reviews_path, user_map, business_map, valid_categories, output_prefix="finaldataset", chunk_size=10000):
    sorted_categories = sorted(valid_categories)
    file_index = 0  # Track number of written files
    
    chunk = []
    for i, line in tqdm(enumerate(open(reviews_path, 'r')), desc="Processing reviews"):
        review = json.loads(line)
        
        # Get profiles
        user_profile = user_map.get(review["user_id"], {})
        biz_profile = business_map.get(review["business_id"], {})

        if not user_profile or not biz_profile:
            continue
        
        # Convert to dense vectors
        user_vec = np.array([user_profile.get(cat, 0) for cat in sorted_categories])
        biz_vec = np.array([biz_profile.get(cat, 0) for cat in sorted_categories])
        
        # Store as list to save memory
        chunk.append({
            "features": np.concatenate([user_vec, biz_vec]).tolist(),
            "target": review["stars"]
        })
        
        # Write chunk to a new Parquet file when chunk size is reached
        if (i + 1) % chunk_size == 0:
            df = pd.DataFrame(chunk)
            output_file = f"{output_prefix}_part{file_index}.parquet"
            df.to_parquet(output_file, engine="pyarrow", compression="snappy", index=False)
            print(f"Saved {output_file}")
            chunk = []  # Clear memory
            file_index += 1  # Increment file index

    # Write remaining data
    if chunk:
        df = pd.DataFrame(chunk)
        output_file = f"{output_prefix}_part{file_index}.parquet"
        df.to_parquet(output_file, engine="pyarrow", compression="snappy", index=False)
        print(f"Saved {output_file}")

# Usage
process_chunks(
    reviews_path=reviews_clean_path,
    user_map=user_profile_map,
    business_map=business_profile_map,
    valid_categories=valid_categories,
    chunk_size=100000
)

Processing reviews: 100456it [00:51, 95.28it/s] 

Saved finaldataset_part0.parquet


Processing reviews: 200612it [01:43, 127.10it/s] 

Saved finaldataset_part1.parquet


Processing reviews: 300612it [02:33, 120.13it/s] 

...

Saved finaldataset_part38.parquet


Processing reviews: 4000589it [34:31, 112.17it/s] 

Saved finaldataset_part39.parquet


Processing reviews: 4097564it [35:07, 1943.98it/s]


Saved finaldataset_part40.parquet


In [25]:
import pyarrow.parquet as pq
import pyarrow as pa
import glob

def merge_parquet_files(output_file="finaldataset.parquet", input_pattern="finaldataset_part*.parquet"):
    # Get list of all partitioned Parquet files
    parquet_files = sorted(glob.glob(input_pattern))

    # Create a Parquet writer to stream data into a single file
    writer = None
    
    for file in parquet_files:
        table = pq.read_table(file)  # Read one file at a time
        if writer is None:
            writer = pq.ParquetWriter(output_file, table.schema, compression="snappy")
        writer.write_table(table)  # Append table to output file
    
    if writer:
        writer.close()

    print(f"Merged all files into {output_file}")

merge_parquet_files()

Merged all files into finaldataset.parquet


In [33]:
parquet_file = pq.ParquetFile("finaldataset.parquet")

batch = next(parquet_file.iter_batches(batch_size=1000))
df_batch = batch.to_pandas()

print(df_batch.head(n=20))

                                             features  target
0   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       3
1   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       5
2   [0.0, 0.0, 0.0, 0.0, 0.0, 0.19297061585725184,...       3
3   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       4
4   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       3
5   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       5
6   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       4
7   [0.0, 0.0, 0.0, 0.0, 0.0, 0.20763672292516025,...       4
8   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       5
9   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       4
10  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       5
11  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       5
12  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       3
13  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       5
14  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...       4
15  [0.0

Despues de concatenar, el archivo parquet final con las 4 millones de filas y 1966 features, pesaba alrededor de 3Gb. Sin embargo, al intentar abrir el archivo directamente con pyarrow, las 27Gb de RAM disponibles colapsan. Lo cual confirma que el peso real del dataset debe ser de más de 30Gb. No entiendo pero es impresionante la compresión que realizan los archivos Parquet. Muy util!